# <center> Assignment 2: HR Analytics - Job Change of Data Scientists </center>

## Data

A company which is active in Big Data and Data Science wants to hire data scientists among people who successfully pass some courses which conduct by the company. Many people signup for their training. Company wants to know which of these candidates are really wants to work for the company after training or looking for a new employment because it helps to reduce the cost and time as well as the quality of training or planning the courses and categorization of candidates. Information related to demographics, education, experience are in hands from candidates signup and enrollment.

This dataset designed to understand the factors that lead a person to leave current job for HR researches too. By model(s) that uses the current credentials,demographics,experience data you will predict the probability of a candidate to look for a new job or will work for the company, as well as interpreting affected factors on employee decision.

The whole data divided to train and test . Target isn't included in test but the test target values data file is in hands for related tasks. A sample submission correspond to enrollee_id of test set provided too with columns : enrollee _id , target

Note:

- The dataset is imbalanced.
- Most features are categorical (Nominal, Ordinal, Binary), some with high cardinality.
- Missing imputation can be a part of your pipeline as well.

Ref: [HR Analytics - Job Change of Data Scientists](https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists)
**Data Descriptions**:

|Features|Descriptions
|-|-|
|enrollee_id| Unique ID for candidate|
|city| City code|
|city_ development _index| Developement index of the city (scaled)|
|gender| Gender of candidate|
|relevent_experience| Relevant experience of candidate|
|enrolled_university| Type of University course enrolled if any|
|education_level| Education level of candidate|
|major_discipline|Education major discipline of candidate|
|experience| Candidate total experience in years|
|company_size| No of employees in current employer's company|
|company_type| Type of current employer|
|lastnewjob| Difference in years between previous job and current job|
|training_hours| training hours completed|
|target| 0 – Not looking for job change, 1 – Looking for a job change|

**Train test split 80/20 random_state 42,KPI: f1-score - 0.75**

In [157]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [158]:
# Load data
job_change = pd.read_csv('train.csv')
job_change.head(5)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [159]:
job_change.shape

(19158, 14)

In [160]:
job_change.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [161]:
job_change.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
target                       0
dtype: int64

In [162]:
job_change.describe()

,enrollee_id,city_development_index,training_hours,target
count,19158.000000,19158.000000,19158.000000,19158.000000
mean,16875.358179,0.828848,65.366896,0.249348
std,9616.292592,0.123362,60.058462,0.432647
min,1.000000,0.448000,1.000000,0.000000
25%,8554.250000,0.740000,23.000000,0.000000
50%,16982.500000,0.903000,47.000000,0.000000
75%,25169.750000,0.920000,88.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


In [163]:
ser = job_change.groupby(['city', 'city_development_index', 'relevent_experience']).experience.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0        >20
1         15
2          5
3         <1
4        >20
        ... 
19153     14
19154     14
19155    >20
19156     <1
19157      2
Name: experience, Length: 19158, dtype: object

In [164]:
job_change['experience'] = ser
job_change.experience.isnull().sum()

0

In [165]:
ser = job_change.groupby(['experience', 'city_development_index', 'relevent_experience']).enrolled_university.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0           no_enrollment
1           no_enrollment
2        Full time course
3           no_enrollment
4           no_enrollment
               ...       
19153       no_enrollment
19154       no_enrollment
19155       no_enrollment
19156       no_enrollment
19157       no_enrollment
Name: enrolled_university, Length: 19158, dtype: object

In [166]:
job_change['enrolled_university'] = ser
job_change.enrolled_university.isnull().sum()

19

In [167]:
ser = job_change.groupby(['enrolled_university', 'city_development_index', 'relevent_experience']).education_level.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0              Graduate
1              Graduate
2              Graduate
3              Graduate
4               Masters
              ...      
19153          Graduate
19154          Graduate
19155          Graduate
19156       High School
19157    Primary School
Name: education_level, Length: 19139, dtype: object

In [168]:
job_change['education_level'] = ser
job_change.education_level.isnull().sum()

23

In [169]:
ser = job_change.groupby(['enrolled_university', 'city_development_index']).education_level.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0              Graduate
1              Graduate
2              Graduate
3              Graduate
4               Masters
              ...      
19153          Graduate
19154          Graduate
19155          Graduate
19156       High School
19157    Primary School
Name: education_level, Length: 19139, dtype: object

In [170]:
job_change['education_level'] = ser
job_change.education_level.isnull().sum()

22

In [171]:
ser = job_change.groupby(['city', 'city_development_index', 'relevent_experience']).education_level.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0              Graduate
1              Graduate
2              Graduate
3              Graduate
4               Masters
              ...      
19153          Graduate
19154          Graduate
19155          Graduate
19156       High School
19157    Primary School
Name: education_level, Length: 19158, dtype: object

In [172]:
job_change['education_level'] = ser
job_change.education_level.isnull().sum()

1

In [173]:
ser = job_change.groupby(['experience', 'relevent_experience']).education_level.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0              Graduate
1              Graduate
2              Graduate
3              Graduate
4               Masters
              ...      
19153          Graduate
19154          Graduate
19155          Graduate
19156       High School
19157    Primary School
Name: education_level, Length: 19158, dtype: object

In [174]:
job_change['education_level'] = ser
job_change.education_level.isnull().sum()

0

In [175]:
ser = job_change.groupby(['experience', 'relevent_experience', 'education_level']).last_new_job.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0            1
1           >4
2        never
3        never
4            4
         ...  
19153        1
19154        4
19155        4
19156        2
19157        1
Name: last_new_job, Length: 19158, dtype: object

In [176]:
job_change['last_new_job'] = ser
job_change.last_new_job.isnull().sum()

3

In [177]:
ser = job_change.groupby(['city', 'city_development_index']).last_new_job.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0            1
1           >4
2        never
3        never
4            4
         ...  
19153        1
19154        4
19155        4
19156        2
19157        1
Name: last_new_job, Length: 19158, dtype: object

In [178]:
job_change['last_new_job'] = ser
job_change.last_new_job.isnull().sum()

0

In [179]:
ser = job_change.groupby(['enrolled_university', 'education_level']).major_discipline.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0                   STEM
1                   STEM
2                   STEM
3        Business Degree
4                   STEM
              ...       
19153         Humanities
19154               STEM
19155               STEM
19156                NaN
19157                NaN
Name: major_discipline, Length: 19139, dtype: object

In [180]:
job_change['major_discipline'] = ser
job_change.major_discipline.isnull().sum()

2364

In [181]:
ser = job_change.groupby(['experience', 'relevent_experience']).major_discipline.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0                   STEM
1                   STEM
2                   STEM
3        Business Degree
4                   STEM
              ...       
19153         Humanities
19154               STEM
19155               STEM
19156               STEM
19157               STEM
Name: major_discipline, Length: 19158, dtype: object

In [182]:
job_change['major_discipline'] = ser
job_change.major_discipline.isnull().sum()

0

In [183]:
ser = job_change.groupby(['experience', 'relevent_experience', 'enrolled_university']).company_size.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0          100-500
1            50-99
2            10/49
3            50-99
4            50-99
           ...    
19153    1000-4999
19154        50-99
19155        50-99
19156      500-999
19157      100-500
Name: company_size, Length: 19139, dtype: object

In [184]:
job_change['company_size'] = ser
job_change.company_size.isnull().sum()

20

In [185]:
ser = job_change.groupby(['experience', 'relevent_experience', 'major_discipline']).company_size.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0          100-500
1            50-99
2            10/49
3            50-99
4            50-99
           ...    
19153    1000-4999
19154        50-99
19155        50-99
19156      500-999
19157      100-500
Name: company_size, Length: 19158, dtype: object

In [186]:
job_change['company_size'] = ser
job_change.company_size.isnull().sum()

0

In [187]:
job_change['company_size'].unique()

array(['100-500', '50-99', '10/49', '<10', '10000+', '5000-9999',
       '1000-4999', '500-999'], dtype=object)

In [188]:
job_change['company_size'].replace({"10/49": "10-49"}, inplace=True)
job_change['company_size'].unique()

array(['100-500', '50-99', '10-49', '<10', '10000+', '5000-9999',
       '1000-4999', '500-999'], dtype=object)

In [189]:
ser = job_change.groupby(['experience', 'company_size', 'major_discipline']).company_type.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0               Pvt Ltd
1               Pvt Ltd
2               Pvt Ltd
3               Pvt Ltd
4        Funded Startup
              ...      
19153     Public Sector
19154           Pvt Ltd
19155           Pvt Ltd
19156           Pvt Ltd
19157           Pvt Ltd
Name: company_type, Length: 19158, dtype: object

In [190]:
job_change['company_type'] = ser
job_change.company_type.isnull().sum()

100

In [191]:
ser = job_change.groupby(['experience', 'city_development_index', 'major_discipline']).company_type.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0               Pvt Ltd
1               Pvt Ltd
2               Pvt Ltd
3               Pvt Ltd
4        Funded Startup
              ...      
19153     Public Sector
19154           Pvt Ltd
19155           Pvt Ltd
19156           Pvt Ltd
19157           Pvt Ltd
Name: company_type, Length: 19158, dtype: object

In [192]:
job_change['company_type'] = ser
job_change.company_type.isnull().sum()

52

In [193]:
ser = job_change.groupby(['enrolled_university', 'education_level', 'major_discipline']).company_type.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0               Pvt Ltd
1               Pvt Ltd
2               Pvt Ltd
3               Pvt Ltd
4        Funded Startup
              ...      
19153     Public Sector
19154           Pvt Ltd
19155           Pvt Ltd
19156           Pvt Ltd
19157           Pvt Ltd
Name: company_type, Length: 19139, dtype: object

In [194]:
job_change['company_type'] = ser
job_change.company_type.isnull().sum()

19

In [195]:
ser = job_change.groupby(['city', 'city_development_index']).gender.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0        Male
1        Male
2        Male
3        Male
4        Male
         ... 
19153    Male
19154    Male
19155    Male
19156    Male
19157    Male
Name: gender, Length: 19158, dtype: object

In [196]:
job_change['gender'] = ser
job_change.gender.isnull().sum()

1

In [197]:
ser = job_change.groupby(['enrolled_university', 'education_level']).gender.apply(lambda x: x.fillna(x.mode()[0]) if len(x.mode())>0 else x.fillna(x.mode()))
ser

0        Male
1        Male
2        Male
3        Male
4        Male
         ... 
19153    Male
19154    Male
19155    Male
19156    Male
19157    Male
Name: gender, Length: 19139, dtype: object

In [198]:
job_change['gender'] = ser
job_change.gender.isnull().sum()

19

In [ ]:
for col in ["gender", "enrolled_university","company_type"]:
    job_change[col] = job_change[col].fillna("Unknown")

In [213]:
job_change.isnull().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
dtype: int64

In [201]:
job_change['experience'] = [27 if x == ">20" else (0 if x == "<1" else int(x)) for x in job_change["experience"]]

In [202]:
job_change['company_size'] = ['1-9' if x == '<10' else ('10000-20000' if x =='10000+' else x) for x in job_change['company_size']]

In [203]:
job_change['last_new_job']  =  [0 if x == 'never' else (6 if x == '>4' else int(x)) for x in job_change['last_new_job']]

In [204]:
job_change["relevent_experience"] = [ 1 if x == 'Has relevent experience' else 0 for x in job_change["relevent_experience"]]

In [205]:
job_change.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,1,no_enrollment,Graduate,STEM,27,100-500,Pvt Ltd,1,36,1.0
1,29725,city_40,0.776,Male,0,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,6,47,0.0
2,11561,city_21,0.624,Male,0,Full time course,Graduate,STEM,5,10-49,Pvt Ltd,0,83,0.0
3,33241,city_115,0.789,Male,0,no_enrollment,Graduate,Business Degree,0,50-99,Pvt Ltd,0,52,1.0
4,666,city_162,0.767,Male,1,no_enrollment,Masters,STEM,27,50-99,Funded Startup,4,8,0.0


In [206]:
scaled_data = pd.get_dummies(job_change)
scaled_data

,enrollee_id,city_development_index,relevent_experience,experience,last_new_job,training_hours,target,city_city_1,city_city_10,city_city_100,...,company_size_50-99,company_size_500-999,company_size_5000-9999,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown
0,8949,0.920,1,27,1,36,1.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,29725,0.776,0,15,6,47,0.0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,11561,0.624,0,5,0,83,0.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,33241,0.789,0,0,0,52,1.0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,666,0.767,1,27,4,8,0.0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,0.878,0,14,1,42,1.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
19154,31398,0.920,1,14,4,52,1.0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
19155,24576,0.920,1,27,4,44,0.0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
19156,5756,0.802,1,0,2,97,0.0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [207]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(scaled_data.drop('target', axis=1), scaled_data['target'], test_size=0.2, random_state=42)

In [214]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs', max_iter=1000)

In [215]:
clf.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [216]:
y_pred = clf.predict(x_test)
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [217]:
from sklearn.metrics import f1_score

In [218]:
job_change_f1 = f1_score(y_test, y_pred)
job_change_f1

0.3801537386443047